# Intro to 2D CNNs
A description and demo notebook to go through creating a 2D CNN and using it with dummy data

## 1. Setting up dummy data

Before learning how to use the cores, let's create a dummy data images. This data will be similar to a batch of images.

Throughout the notebook we will refer to the elements of this shape in the following manner:

[1] is the number of channels (can be input, hidden, output)

[144] is the height of image or feature maps

[256] is the height of image or feature maps

[32] is the batch size, which is not as relevant for understanding the material in this notebook.

In [1]:
# To access to neuropixel_predictor
import sys
sys.path.append('../')

# Basic imports
import warnings
import random

# Essential imports
import numpy as np
import torch

In [2]:
warnings.filterwarnings("ignore", category=UserWarning)
device = "cuda" if torch.cuda.is_available() else "cpu"
random_seed = 42

## 2. Setting Up the data

In [3]:
images = torch.ones(32, 1, 144, 256)

## 3. Using Stacked 2D Core

In [29]:
stacked2dcore_config = {
    # core args
    # 'input_channels': 1,
    'input_kern': 7,
    'hidden_kern': 5,
    'hidden_channels': 64,
    'layers': 3,
    'stack': -1,
    'pad_input': True,
    'gamma_input': 6.3831
}

In [5]:
# from neuralpredictors.layers.cores import Stacked2dCore
# from neuralpredictors.utils import get_module_output

# core = Stacked2dCore(input_channels=1,
#                      hidden_channels=64,
#                      input_kern=9,
#                      hidden_kern=7)

# in_shape_dict = {k: get_module_output(core, in_shape)[1:] for k, in_shape in in_shapes_dict.items()}



from neuropixel_predictor.layers.cores import Stacked2dCore 

stacked2d_core = Stacked2dCore(**stacked2dcore_config)
stacked2d_core

# stacked2dcore_out = stacked2d_core(images)
# print(stacked2dcore_out.shape)

Stacked2dCore(
  (_input_weights_regularizer): LaplaceL2(
    (laplace): Laplace()
  )
  (features): Sequential(
    (layer0): Sequential(
      (conv): Conv2d(1, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
      (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (nonlin): AdaptiveELU()
    )
    (layer1): Sequential(
      (conv): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (nonlin): AdaptiveELU()
    )
    (layer2): Sequential(
      (conv): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (nonlin): AdaptiveELU()
    )
  )
) [Stacked2dCore regularizers: gamma_hidden = 0|gamma_input = 6.3831|skip = 0]

In [6]:
# Trying out on images
stacked2dcore_out = stacked2d_core(images)
print(stacked2dcore_out.shape)

torch.Size([32, 64, 144, 256])


## 3. Using Readout to attach Factorized Readout

In [7]:
from neuropixel_predictor.layers.readouts import FullFactorized2d, MultiReadoutBase

In [8]:
in_shapes_dict = {
    '21067-10-18': torch.Size([64, 144, 256]),
    '22846-10-16': torch.Size([64, 144, 256])
}

n_neurons_dict = {'21067-10-18': 8372, '22846-10-16': 7344}

In [9]:
factorized_readout = MultiReadoutBase(
    in_shape_dict=in_shapes_dict,
    n_neurons_dict=n_neurons_dict,
    base_readout=FullFactorized2d,
    bias=True,
)

## 4. Invoke core and readout (dummy data)

In [10]:
# Forward pass with core
core_output = stacked2d_core(images)

# Forward pass with readout
readout_output_sample = factorized_readout(core_output, data_key="21067-10-18")

readout_output_sample

tensor([[-0.0016,  0.0006,  0.0014,  ...,  0.0053, -0.0108, -0.0069],
        [-0.0016,  0.0006,  0.0014,  ...,  0.0053, -0.0108, -0.0069],
        [-0.0016,  0.0006,  0.0014,  ...,  0.0053, -0.0108, -0.0069],
        ...,
        [-0.0016,  0.0006,  0.0014,  ...,  0.0053, -0.0108, -0.0069],
        [-0.0016,  0.0006,  0.0014,  ...,  0.0053, -0.0108, -0.0069],
        [-0.0016,  0.0006,  0.0014,  ...,  0.0053, -0.0108, -0.0069]],
       grad_fn=<AddBackward0>)

## 5. Testing with real data (test data)
Utility functions from https://github.com/sinzlab/sensorium

In [11]:
%%capture 
!pip install git+https://github.com/sinzlab/sensorium.git

Defining some helper functions to extract the data

In [12]:
# The following are minimal adaptations of three utility functions found in nnfabrik that we need to initialise
# the core and readouts later on.

def get_data(dataset_fn, dataset_config):
    """
    See https://github.com/sinzlab/nnfabrik/blob/5b6e7379cb5724a787cdd482ee987b8bc0dfacf3/nnfabrik/builder.py#L87
    for the original implementation and documentation if you are interested.
    """
    return dataset_fn(**dataset_config)

def get_dims_for_loader_dict(dataloaders):
    """
    See https://github.com/sinzlab/nnfabrik/blob/5b6e7379cb5724a787cdd482ee987b8bc0dfacf3/nnfabrik/utility/nn_helpers.py#L39
    for the original implementation and docstring if you are interested.
    """
    
    def get_io_dims(data_loader):
        items = next(iter(data_loader))
        if hasattr(items, "_asdict"):  # if it's a named tuple
            items = items._asdict()

        if hasattr(items, "items"):  # if dict like
            return {k: v.shape for k, v in items.items()}
        else:
            return (v.shape for v in items)

    return {k: get_io_dims(v) for k, v in dataloaders.items()}


def set_random_seed(seed: int, deterministic: bool = True):
    """
    See https://github.com/sinzlab/nnfabrik/blob/5b6e7379cb5724a787cdd482ee987b8bc0dfacf3/nnfabrik/utility/nn_helpers.py#L53
    for the original implementation and docstring if you are intereseted.
    """
    random.seed(seed)
    np.random.seed(seed)
    if deterministic:
        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)  # this sets both CPU and CUDA seeds for PyTorch

Loading the data

In [17]:
## Load the data: you can modify this if you have stored it in another location
from sensorium.datasets import static_loaders

DATA_PATH = '/Users/tarek/Documents/UNI/Lab Rotations/Kremkow/Data/Test/'

filenames = [
    DATA_PATH + 'static21067-10-18-GrayImageNet-94c6ff995dac583098847cfecd43e7b6.zip', 
    DATA_PATH + 'static22846-10-16-GrayImageNet-94c6ff995dac583098847cfecd43e7b6.zip'
    ]

dataset_config = {'paths': filenames,
                 'normalize': True,
                 'include_behavior': False,
                 'include_eye_position': True,
                 'batch_size': 32,
                 'scale':1,
                 'cuda': True if device == 'cuda' else False,
                 }

dataloaders = get_data(dataset_fn=static_loaders, dataset_config=dataset_config)

Process the data step by step

In [32]:
# We only need the train dataloaders to extract the session keys (could also use test or validation for this)
train_dataloaders = dataloaders["train"]

# Obtain the named tuple fields from the first entry of the first dataloader in the dictionary
example_batch = next(iter(list(train_dataloaders.values())[0]))
in_name, out_name = (
    list(example_batch.keys())[:2] if isinstance(example_batch, dict) else example_batch._fields[:2]
)

session_shape_dict = get_dims_for_loader_dict(train_dataloaders)
input_channels = [v[in_name][1] for v in session_shape_dict.values()]

core_input_channels = (
    list(input_channels.values())[0]
    if isinstance(input_channels, dict)
    else input_channels[0]
)

**Core: Setting up**

In [33]:
set_random_seed(random_seed)
core = Stacked2dCore(
    input_channels=core_input_channels,
    **stacked2dcore_config,
)
core

Stacked2dCore(
  (_input_weights_regularizer): LaplaceL2(
    (laplace): Laplace()
  )
  (features): Sequential(
    (layer0): Sequential(
      (conv): Conv2d(1, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
      (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (nonlin): AdaptiveELU()
    )
    (layer1): Sequential(
      (conv): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (nonlin): AdaptiveELU()
    )
    (layer2): Sequential(
      (conv): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (nonlin): AdaptiveELU()
    )
  )
) [Stacked2dCore regularizers: gamma_hidden = 0|gamma_input = 6.3831|skip = 0]

**Core: Example forward pass**